## Desafío 3 - DH - ML Clasificación

<a id="tabla_contenidos"></a> 
### Tabla de Contenidos

#### <a href='#section_proposito'>0. Propósito de esta Notebook</a>
- ##### <a href='#section_motivacion'>0.1 Motivación</a>
- ##### <a href='#section_dataset_info'>0.2 Detalles del Dataset</a>
- ##### <a href='#section_antecedentes'>0.3 Estado del Arte</a>

#### <a href='#section_preparacion'>1. Preparación del Dataset</a>
- ##### <a href='#section_importacion'>1.1 Importación</a>
- ##### <a href='#section_geografia'>1.2 Exploración geográfica</a>
- ##### <a href='#section_preguntas'>1.3 Análisis por columnas</a>
- ##### <a href='#section_eliminacion'>1.4 Acotación de interés</a>
- ##### <a href='#section_split'>1.5 Selección de Features y target</a>

#### <a href='#section_algoritmos'>2. Algoritmos y predicción</a>
- ##### <a href='#knn'>2.1 K Nearest Neighbors (KNN)</a>
- ##### <a href='#reg_log'>2.2 Regresión Logística</a>
- ##### <a href='#naive'>2.3 Naive Bayes</a>
- ##### <a href='#naive_gaussian'>2.4 Naive Bayes Gaussian</a>

#### <a href='#section_consolidacion'>3. Consolidación</a>
- ##### <a href='#pipe'>3.1 Pipelines</a>
- ##### <a href='#conclusion'>3.2 Conclusión</a>

### <a id='section_proposito'>0. Propósito de esta Notebook</a>

#### <a id="section_motivacion">0.1 Motivación</a> 

<a id='section_preparacion'></a> 
### 1. Preparación del Dataset

#### <a id="section_importacion">1.1 Importación de datos</a> 

##### Importación de datos de población y resultados de encuesta

In [11]:
# librerías
import numpy as np
import pandas as pd

In [12]:
resultados_del_test = pd.read_csv("data/resultados-del-test.csv", delimiter = ",")

In [13]:
poblacion_por_barrio = pd.read_csv("data/caba_pob_barrios_2010.csv")

##### Unifico en un dataset

In [14]:
def elimina_tildes(cadena):
    s = ''.join((c for c in unicodedata.normalize('NFD',cadena) if unicodedata.category(c) != 'Mn'))
    return s

In [15]:
poblacion_por_barrio.BARRIO = poblacion_por_barrio.BARRIO.apply(elimina_tildes).apply(lambda barrio: barrio.capitalize())

NameError: name 'unicodedata' is not defined

In [16]:
resultados_del_test.barrio = resultados_del_test.barrio.apply(elimina_tildes).apply(lambda barrio: barrio.capitalize())

NameError: name 'unicodedata' is not defined

In [17]:
poblacion_por_barrio.columns = ['barrio', 'poblacion']

In [18]:
poblacion_por_barrio = poblacion_por_barrio.set_index('barrio')

In [19]:
def obtener_poblacion_por_barrio(barrio):
    return poblacion_por_barrio.loc[barrio].poblacion

In [20]:
def imputar_barrio(barrio):
    barrios = poblacion_por_barrio.index.tolist()
    if barrio == "La boca": return "Boca"
    if barrio == "Montserrat": return "Monserrat"
    if barrio == "Villa general mitre": return "Villa gral. mitre"
    if barrio == "La paternal": return "Paternal"
    if barrio == "Otro": return np.random.choice(barrios)
    return barrio

In [21]:
resultados_del_test["barrio_completo"] = resultados_del_test.barrio.apply(imputar_barrio)

In [22]:
resultados_del_test["poblacion_por_barrio"] = resultados_del_test.barrio_completo.apply(obtener_poblacion_por_barrio)

KeyError: 'Constitución'

##### Exporto el dataset unificado

In [ ]:
resultados_del_test.to_csv("data/dataset_joined.csv")

In [ ]:
resultados_del_test = pd.read_csv("data/resultados-del-test.csv", delimiter = ",")

#### <a id="section_geografia">1.2 Exploración geográfica</a> 
